# GitVizz Graph Visualization Demo

This notebook demonstrates the new graph loading, saving, and visualization features of GitVizz.

In [1]:
import sys
sys.path.append('..')

from gitvizz import GraphGenerator, IPYSIGMA_AVAILABLE
import zipfile
import os
import tempfile
import urllib.request
from pathlib import Path

# GitHub zip URL for the example repository
GITHUB_ZIP_URL = "https://github.com/adithya-s-k/omniparse/archive/refs/heads/main.zip"

print(f"ipysigma available: {IPYSIGMA_AVAILABLE}")

ipysigma available: True


## 1. NEW: Load and Analyze Repository Directly from ZIP or Folder

GitVizz now supports loading directly from ZIP files or directories with the convenient `from_source()` method!

In [2]:
# NEW: Create GraphGenerator directly from ZIP file or directory!
# No more manual extraction and file loading required - now with dynamic GitHub download!

# Create a temporary file to store the downloaded zip
with tempfile.NamedTemporaryFile(suffix='.zip', delete=False) as temp_zip:
    try:
        print(f"📦 Downloading ZIP from GitHub...")
        urllib.request.urlretrieve(GITHUB_ZIP_URL, temp_zip.name)
        zip_path = temp_zip.name
        print(f"✅ Successfully downloaded to: {zip_path}")
        
        # Create generator directly from ZIP - much simpler!
        generator = GraphGenerator.from_source(
            zip_path,
            # max_files=25,  # Limit files for demo performance
            file_extensions=['.py']  # Only Python files for this demo
        )
        
        # Generate and analyze
        graph_data = generator.generate()
        print("✅ Analysis complete:")
        print(f"   - Nodes: {len(graph_data['nodes'])}")
        print(f"   - Edges: {len(graph_data['edges'])}")
        print(f"   - Project type: {generator.project_type}")
        
    except Exception as e:
        print(f"❌ Failed to process ZIP: {e}")
    finally:
        # Clean up the temporary zip file
        if os.path.exists(zip_path):
            os.unlink(zip_path)
            print("✅ Cleaned up temporary zip file")

📦 Downloading ZIP from GitHub...
✅ Successfully downloaded to: /var/folders/m4/wjv39knd26bdfj61bssrwj3c0000gn/T/tmpnfb4bzex.zip
Loaded 30 files from /var/folders/m4/wjv39knd26bdfj61bssrwj3c0000gn/T/tmpnfb4bzex.zip
GraphGenerator: Determined project root: /private/var/folders/m4/wjv39knd26bdfj61bssrwj3c0000gn/T
Identified project type: python
✅ Analysis complete:
   - Nodes: 509
   - Edges: 1159
   - Project type: python
✅ Cleaned up temporary zip file


In [8]:
# # Demonstrate directory loading as well
# print("\\n📁 Loading from directory example:")

# # Load from a directory instead of ZIP
# try:
#     generator_from_dir = GraphGenerator.from_source(
#         '../gitvizz',  # Load the GitVizz package itself
#         file_extensions=['.py'],
#         ignore_patterns=[
#             # '**/__pycache__/**',
#             # '**/.*',  # Hidden files
#             # '**/test_*',  # Test files
#             # '**/*.pyc'
#         ],
#         # max_files=5  # Keep it small for demo
#     )
    
#     dir_graph = generator_from_dir.generate()
#     print(f"✅ Directory analysis:")
#     print(f"   - Nodes: {len(dir_graph['nodes'])}")
#     print(f"   - Edges: {len(dir_graph['edges'])}")
#     print(f"   - Project type: {generator_from_dir.project_type}")
    
# except Exception as e:
#     print(f"❌ Directory loading failed: {e}")

# print("\\n🎉 The from_source() method makes GitVizz much easier to use!")

## 3. Save and Load Graph in Different Formats

In [11]:
# Save graph in JSON format
generator.save_json("graph_data.json")
print("Saved graph to graph_data.json")

# Save graph in GraphML format
generator.save_graphml("graph_data.graphml")
print("Saved graph to graph_data.graphml")

Saved graph to graph_data.json
Saved graph to graph_data.graphml


In [12]:
# Test loading from JSON
new_generator_json = GraphGenerator([])
new_generator_json.load_json("graph_data.json")
print(f"Loaded from JSON: {len(new_generator_json.all_nodes_data)} nodes, {len(new_generator_json.all_edges_data)} edges")

# Test loading from GraphML
new_generator_graphml = GraphGenerator([])
new_generator_graphml.load_graphml("graph_data.graphml")
print(f"Loaded from GraphML: {len(new_generator_graphml.all_nodes_data)} nodes, {len(new_generator_graphml.all_edges_data)} edges")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
Loaded from JSON: 509 nodes, 1159 edges
GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
Loaded from GraphML: 509 nodes, 781 edges


## 4. Convert to NetworkX

In [13]:
# Convert to NetworkX graph
import networkx as nx
nx_graph = generator.to_networkx()
print(f"NetworkX graph: {nx_graph.number_of_nodes()} nodes, {nx_graph.number_of_edges()} edges")
print(f"Graph type: {type(nx_graph)}")

# Show some basic NetworkX analysis
print("Graph properties:")
print(f"- Is directed: {nx_graph.is_directed()}")
print(f"- Number of weakly connected components: {nx.number_weakly_connected_components(nx_graph)}")

# Show top 5 nodes by degree
degrees = dict(nx_graph.degree())
top_nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:5]
print("Top 5 nodes by degree:")
for node_id, degree in top_nodes:
    node_data = nx_graph.nodes[node_id]
    print(f"- {node_data.get('name', node_id)} ({node_data.get('category', 'unknown')}): {degree}")

NetworkX graph: 509 nodes, 781 edges
Graph type: <class 'networkx.classes.digraph.DiGraph'>
Graph properties:
- Is directed: True
- Number of weakly connected components: 1
Top 5 nodes by degree:
- get_content_of_website (function): 31
- demo (module): 30
- router (module): 26
- utils (module): 19
- model_loader (module): 18


## 5. Interactive Visualization with ipysigma

In [15]:
# Check if ipysigma is available
if IPYSIGMA_AVAILABLE:
    print("ipysigma is available - creating interactive visualization")
    
    # Create interactive visualization
    sigma_widget = generator.visualize(
        width=1000,
        height=700
    )
    
    # Display the widget
    display(sigma_widget)
else:
    print("ipysigma is not available. Install it with: pip install ipysigma")
    print("Falling back to basic NetworkX visualization")
    
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 8))
    
    # Simple NetworkX layout
    pos = nx.spring_layout(nx_graph, k=1, iterations=50)
    
    # Draw nodes colored by category
    categories = set()
    for node_id in nx_graph.nodes():
        categories.add(nx_graph.nodes[node_id].get('category', 'unknown'))
    
    colors = plt.cm.Set3(range(len(categories)))
    category_color_map = dict(zip(categories, colors))
    
    for category in categories:
        nodes_in_category = [n for n in nx_graph.nodes() 
                           if nx_graph.nodes[n].get('category') == category]
        nx.draw_networkx_nodes(nx_graph, pos, 
                              nodelist=nodes_in_category,
                              node_color=[category_color_map[category]],
                              label=category, node_size=100)
    
    nx.draw_networkx_edges(nx_graph, pos, alpha=0.3, arrows=True)
    plt.legend()
    plt.title("Repository Dependency Graph")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

ipysigma is available - creating interactive visualization


Sigma(nx.DiGraph with 509 nodes and 781 edges)

## 6. Advanced ipysigma Customization (if available)

In [16]:
if IPYSIGMA_AVAILABLE:
    # Custom visualization with degree-based sizing
    degrees = dict(nx_graph.degree())
    
    sigma_custom = generator.visualize(
        node_size=degrees,  # Size nodes by degree
        node_color='category',  # Color by category (default)
        width=1200,
        height=800
    )
    
    display(sigma_custom)
else:
    print("ipysigma not available for custom visualization")

Sigma(nx.DiGraph with 509 nodes and 781 edges)

## 7. File Format Interoperability Test

In [17]:
# Test round-trip conversion: Original -> JSON -> NetworkX -> GraphML -> Back to GitVizz

# 1. Save original to JSON
generator.save_json("test_original.json")

# 2. Load from JSON into new instance
gen_from_json = GraphGenerator([])
gen_from_json.load_json("test_original.json")

# 3. Convert to NetworkX and save as GraphML
nx_from_json = gen_from_json.to_networkx()
gen_from_json.save_graphml("test_roundtrip.graphml")

# 4. Load GraphML back into new instance
gen_from_graphml = GraphGenerator([])
gen_from_graphml.load_graphml("test_roundtrip.graphml")

# 5. Compare results
print("Round-trip test results:")
print(f"Original: {len(generator.all_nodes_data)} nodes, {len(generator.all_edges_data)} edges")
print(f"From JSON: {len(gen_from_json.all_nodes_data)} nodes, {len(gen_from_json.all_edges_data)} edges")
print(f"From GraphML: {len(gen_from_graphml.all_nodes_data)} nodes, {len(gen_from_graphml.all_edges_data)} edges")

# Cleanup test files
for file in ["test_original.json", "test_roundtrip.graphml"]:
    if os.path.exists(file):
        os.remove(file)
        print(f"Cleaned up {file}")

GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
GraphGenerator: Determined project root: /Users/adithyaskolavi/projects/git-repo-mcp/gitvizz/examples
Identified project type: unknown
Round-trip test results:
Original: 509 nodes, 1159 edges
From JSON: 509 nodes, 1159 edges
From GraphML: 509 nodes, 781 edges
Cleaned up test_original.json
Cleaned up test_roundtrip.graphml
